# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

In [1]:
# Imports
import pandas as pd
import numpy as np

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [19]:
recipes = pd.read_csv("recipes.tsv", sep="\t")
recipes

,recipe_slug,recipe_title,recipe_introduction
0,falafel,Falafel,Falafel is a popular Middle Eastern dish made ...
1,spamburger,Spamburger,Spamburger is a type of hamburger that is made...
2,bacon-fried-rice,Bacon Fried Rice,Bacon fried rice is a savory and satisfying di...
3,chicken-fingers,Chicken Fingers,Chicken fingers are a popular dish made from c...
4,apple-crisp,Apple Crisp,Apple crisp is a classic dessert made with bak...
...,...,...,...
95,bacon-mac-and-cheese,Bacon Mac And Cheese,Bacon mac and cheese is a delicious and comfor...
96,chicken-alfredo-lasagna,Chicken Alfredo Lasagna,Chicken alfredo lasagna is a delicious twist o...
97,classic-beef-lasagna,Classic Beef Lasagna,Classic beef lasagna is a hearty and comfortin...
98,vegetarian-mushroom-lasagna,Vegetarian Mushroom Lasagna,Vegetarian mushroom lasagna is a delicious and...


In [38]:
ratings = pd.read_csv("recipes_rated.csv")
ratings = ratings[["recipe_slug", "ratings_raw"]]
ratings

,recipe_slug,ratings_raw
0,falafel,5
1,spamburger,3
2,bacon-fried-rice,3
3,chicken-fingers,1
4,apple-crisp,4
...,...,...
95,bacon-mac-and-cheese,3
96,chicken-alfredo-lasagna,5
97,classic-beef-lasagna,3
98,vegetarian-mushroom-lasagna,2


In [39]:
ratings["rating"] = (ratings["ratings_raw"] - 1) * 0.25

In [40]:
ratings = ratings.drop("ratings_raw", axis=1)
ratings

,recipe_slug,rating
0,falafel,1.00
1,spamburger,0.50
2,bacon-fried-rice,0.50
3,chicken-fingers,0.00
4,apple-crisp,0.75
...,...,...
95,bacon-mac-and-cheese,0.50
96,chicken-alfredo-lasagna,1.00
97,classic-beef-lasagna,0.50
98,vegetarian-mushroom-lasagna,0.25


In [42]:
ratings.to_csv("ratings.tsv", sep="\t", index=False)

Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [58]:
# YOUR CHANGES HERE

recipe_tags = pd.read_csv("recipe-tags.tsv", sep="\t")
recipe_tags


,recipe_slug,recipe_tag
0,spam-musubi,hawaiian
1,spam-musubi,nori
2,spam-musubi,onthego
3,spam-musubi,rice
4,spam-musubi,snack
...,...,...
747,bacon-souffle,breakfast
748,bacon-souffle,brunch
749,bacon-souffle,cheese
750,bacon-souffle,eggs


In [72]:
encoded_recipe_tags = recipe_tags.pivot_table(
    index="recipe_slug", 
    columns="recipe_tag", 
    aggfunc=lambda x: 1,
    fill_value=0
)
encoded_recipe_tags

recipe_tag,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,avocado,bacon,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
recipe_slug,,,,,,,,,,,,,,,,,,,,,
almond-chip-cookies,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
almond-croissants,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
apple-crisp,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
apple-crumble,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
apple-pie,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tempura-udon,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
udon,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
vanilla-souffle,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [78]:
features = pd.merge(ratings[["recipe_slug"]], encoded_recipe_tags, how="left", on=["recipe_slug"])
features

,recipe_slug,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,avocado,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bacon-mac-and-cheese,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,chicken-alfredo-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,classic-beef-lasagna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,vegetarian-mushroom-lasagna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
features.insert(1, "bias", 1)
features

,recipe_slug,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bacon-mac-and-cheese,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,chicken-alfredo-lasagna,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,classic-beef-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,vegetarian-mushroom-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [80]:
features.iloc[:, 2:] = features.iloc[:, 2:].astype(int)
features

,recipe_slug,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bacon-mac-and-cheese,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,chicken-alfredo-lasagna,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,classic-beef-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,vegetarian-mushroom-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [81]:
features.to_csv("features.tsv", sep="\t", index=False)

Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [ ]:
# YOUR CHANGES HERE

...

Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not include the bias.

In [ ]:
# YOUR CHANGES HERE

...

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [ ]:
# YOUR CHANGES HERE

...

Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [ ]:
# YOUR CHANGES HERE

...

Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [ ]:
# YOUR CHANGES HERE

...

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.